In [1]:
import sys, os, time, getpass

sys.path.append("/home/tatiane/lib/")

import pessoal
from pessoal import *
print('AppID: ', sc.applicationId)

# Pedir ao spark para exibir só erros, escondendo avisos (WARN) e infos
# O correto é resolver a causa como, por exemplo: ordenados = Window.partitionBy().orderBy(dfs[i].columns[0])
spark.sparkContext.setLogLevel("ERROR")

Tempo inicial da execucao: 2025-06-30 17:26:33.624612
User: tatiane
Node: Inspiron-3583
AppID:  local-1751315180547


# Censo escolar 2023

In [2]:
# Leitura com encoding apropriado para o conteúdo da base
censo = spark.read.csv('/home/tatiane/Documentos/censo_escolar2/projeto_2/microdados_censo_escolar_2023/dados/microdados_ed_basica_2023.csv', header = True, multiLine = True, sep = ';', encoding="latin1")
write_path = '/home/tatiane/Documentos/censo_escolar2/projeto_2/analise3/'

In [3]:
pessoal.completudeSchema(censo)

Qtd. registros: 217625 | Quantidade de colunas:  408
root
 |-- NU_ANO_CENSO: string (nullable = true)
 |-- NO_REGIAO: string (nullable = true)
 |-- CO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CO_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- CO_MUNICIPIO: string (nullable = true)
 |-- NO_REGIAO_GEOG_INTERM: string (nullable = true)
 |-- CO_REGIAO_GEOG_INTERM: string (nullable = true)
 |-- NO_REGIAO_GEOG_IMED: string (nullable = true)
 |-- CO_REGIAO_GEOG_IMED: string (nullable = true)
 |-- NO_MESORREGIAO: string (nullable = true)
 |-- CO_MESORREGIAO: string (nullable = true)
 |-- NO_MICRORREGIAO: string (nullable = true)
 |-- CO_MICRORREGIAO: string (nullable = true)
 |-- CO_DISTRITO: string (nullable = true)
 |-- NO_ENTIDADE: string (nullable = true)
 |-- CO_ENTIDADE: string (nullable = true)
 |-- TP_DEPENDENCIA: string (nullable = true)
 |-- TP_CATEGORIA_ESCOLA_PRIVADA: string (nullab

In [4]:
censo = createId(censo, 'id_unico')
censo.select(F.min(F.col('id_unico')), F.max(F.col('id_unico'))).show()

+-------------+-------------+
|min(id_unico)|max(id_unico)|
+-------------+-------------+
|            1|       217625|
+-------------+-------------+



### Transformação dos dados

In [5]:
analise3 = censo.withColumn('tipo_vinculo', F.when(F.col('IN_VINCULO_SECRETARIA_EDUCACAO') == 1, 'secretaria da educacao').\
                         when(F.col('IN_VINCULO_SEGURANCA_PUBLICA') == 1, 'seguranca publica').\
                         when(F.col('IN_VINCULO_SECRETARIA_SAUDE') == 1, 'secretaria da saúde').\
                         when(F.col('IN_VINCULO_OUTRO_ORGAO') == 1, 'outro orgão').otherwise('Não aplicavel para escolas particulares'))

In [6]:
analise3.groupBy('tipo_vinculo').count().show(truncate=False)

print('Distribuição original do Órgão ao qual a escola pública está vinculada:')
for col in censo.columns:
    if col.startswith('IN_VINCULO_'):
        censo.groupBy(col).count().show()

# O count abaixo é apenas referente a um conjunto de combinações com o vinculo das escolas. A quantidade pode ser maior ou menor a depender das combinações.
print('É possível possuir mais de um tipo de vinculo.')
print('Quantidade de registros marcados como vinculo com a secretaria da educação e outros: ', censo.filter((F.col('IN_VINCULO_SECRETARIA_EDUCACAO') == 1) & ((F.col('IN_VINCULO_SEGURANCA_PUBLICA') == 1) | (F.col('IN_VINCULO_SECRETARIA_SAUDE') == 1) | (F.col('IN_VINCULO_OUTRO_ORGAO') == 1))).count())

+---------------------------------------+------+
|tipo_vinculo                           |count |
+---------------------------------------+------+
|outro orgão                            |335   |
|secretaria da educacao                 |137540|
|secretaria da saúde                    |5     |
|seguranca publica                      |34    |
|Não aplicavel para escolas particulares|79711 |
+---------------------------------------+------+

Distribuição original do Órgão ao qual a escola pública está vinculada:


+------------------------------+------+
|IN_VINCULO_SECRETARIA_EDUCACAO| count|
+------------------------------+------+
|                             0|   374|
|                          null| 79711|
|                             1|137540|
+------------------------------+------+



+----------------------------+------+
|IN_VINCULO_SEGURANCA_PUBLICA| count|
+----------------------------+------+
|                           0|137684|
|                        null| 79711|
|                           1|   230|
+----------------------------+------+



+---------------------------+------+
|IN_VINCULO_SECRETARIA_SAUDE| count|
+---------------------------+------+
|                          0|137894|
|                       null| 79711|
|                          1|    20|
+---------------------------+------+



+----------------------+------+
|IN_VINCULO_OUTRO_ORGAO| count|
+----------------------+------+
|                     0|137524|
|                  null| 79711|
|                     1|   390|
+----------------------+------+

É possível possuir mais de um tipo de vinculo.


Quantidade de registros marcados como vinculo com a secretaria da educação e outros:  259


In [7]:
analise3 = analise3.withColumn('local_func_escola', F.when(F.col('IN_LOCAL_FUNC_PREDIO_ESCOLAR') == 1, 'Prédio Escolar').\
                         when(F.col('IN_LOCAL_FUNC_SALAS_OUTRA_ESC') == 1, 'Salas em outra escola').\
                         when(F.col('IN_LOCAL_FUNC_SOCIOEDUCATIVO') == 1, 'Unidade de Atendimento socioeducativo').\
                         when(F.col('IN_LOCAL_FUNC_UNID_PRISIONAL') == 1, 'Unidade Prisional').\
                         when(F.col('IN_LOCAL_FUNC_PRISIONAL_SOCIO') == 1, 'Unidade Prisional ou Unidade de atendimento socioeducativo').\
                         when(F.col('IN_LOCAL_FUNC_GALPAO') == 1, 'Galpão/Rancho/Paiol/Barracão').\
                         when(F.col('IN_LOCAL_FUNC_OUTROS') == 1, 'Outro').otherwise('Sem informação'))

In [8]:
analise3.groupBy('local_func_escola').count().show(truncate=False)

print('Distribuição original do Local de funcionamento da escola:')
for col in censo.columns:
    if col.startswith('IN_LOCAL_FUNC'):
        censo.groupBy(col).count().show()
        
# O count abaixo é apenas referente a um conjunto de combinações com o local de funcionamento. A quantidade pode ser maior ou menor a depender das combinações.
print('É possível possuir mais de um local de funcionamento.')
print('Quantidade de registros marcados com mais de um local de funcionamento: ', censo.filter((F.col('IN_LOCAL_FUNC_PREDIO_ESCOLAR') == 1) & ((F.col('IN_LOCAL_FUNC_SALAS_OUTRA_ESC') == 1) | (F.col('IN_LOCAL_FUNC_SOCIOEDUCATIVO') == 1) | (F.col('IN_LOCAL_FUNC_UNID_PRISIONAL') == 1) | (F.col('IN_LOCAL_FUNC_PRISIONAL_SOCIO') == 1) | (F.col('IN_LOCAL_FUNC_GALPAO') == 1) | (F.col('IN_LOCAL_FUNC_OUTROS') == 1))).count())

+-------------------------------------+------+
|local_func_escola                    |count |
+-------------------------------------+------+
|Unidade Prisional                    |236   |
|Galpão/Rancho/Paiol/Barracão         |1342  |
|Unidade de Atendimento socioeducativo|53    |
|Salas em outra escola                |685   |
|Prédio Escolar                       |176052|
|Sem informação                       |37395 |
|Outro                                |1862  |
+-------------------------------------+------+

Distribuição original do Local de funcionamento da escola:


+----------------------------+------+
|IN_LOCAL_FUNC_PREDIO_ESCOLAR| count|
+----------------------------+------+
|                           0|  4178|
|                        null| 37395|
|                           1|176052|
+----------------------------+------+



+----------------------------+------+
|IN_LOCAL_FUNC_SOCIOEDUCATIVO| count|
+----------------------------+------+
|                           0|179997|
|                        null| 37395|
|                           1|   233|
+----------------------------+------+



+----------------------------+------+
|IN_LOCAL_FUNC_UNID_PRISIONAL| count|
+----------------------------+------+
|                           0|179610|
|                        null| 37395|
|                           1|   620|
+----------------------------+------+



+-----------------------------+------+
|IN_LOCAL_FUNC_PRISIONAL_SOCIO| count|
+-----------------------------+------+
|                            0|179405|
|                         null| 37395|
|                            1|   825|
+-----------------------------+------+



+--------------------+------+
|IN_LOCAL_FUNC_GALPAO| count|
+--------------------+------+
|                   0|177755|
|                null| 37395|
|                   1|  2475|
+--------------------+------+



+-----------------------------+------+
|IN_LOCAL_FUNC_SALAS_OUTRA_ESC| count|
+-----------------------------+------+
|                            0|174513|
|                         null| 37395|
|                            1|  5717|
+-----------------------------+------+



+--------------------+------+
|IN_LOCAL_FUNC_OUTROS| count|
+--------------------+------+
|                   0|169061|
|                null| 37395|
|                   1| 11169|
+--------------------+------+

É possível possuir mais de um local de funcionamento.


Quantidade de registros marcados com mais de um local de funcionamento:  14974


In [9]:
analise3 = analise3.withColumn('tipo_agua', F.when(F.col('IN_AGUA_REDE_PUBLICA') == 1, 'Rede pública').\
                         when(F.col('IN_AGUA_POCO_ARTESIANO') == 1, 'Poço artesiano').\
                         when(F.col('IN_AGUA_CACIMBA') == 1, 'Cacimba/Cisterna/Poço').\
                         when(F.col('IN_AGUA_FONTE_RIO') == 1, 'Fonte/Rio/Igarapé/Riacho/Córrego').\
                         when(F.col('IN_AGUA_INEXISTENTE') == 1, 'Não há abastecimento de água').otherwise('Sem informação'))

In [10]:
analise3.groupBy('tipo_agua').count().show(truncate=False)

print('Distribuição original do Abastecimento de água:')
for col in censo.columns:
    if col.startswith('IN_AGUA'):
        censo.groupBy(col).count().show()
        
# O count abaixo é apenas referente a um conjunto de combinações com o local de abastecimento. A quantidade pode ser maior ou menor a depender das combinações.
print('É possível possuir mais de um local de abastecimento.')
print('Quantidade de registros marcados com mais de um local de abastecimento: ', censo.filter((F.col('IN_AGUA_REDE_PUBLICA') == 1) & ((F.col('IN_AGUA_POCO_ARTESIANO') == 1) | (F.col('IN_AGUA_CACIMBA') == 1) | (F.col('IN_AGUA_FONTE_RIO') == 1))).count())

+--------------------------------+------+
|tipo_agua                       |count |
+--------------------------------+------+
|Não há abastecimento de água    |3063  |
|Fonte/Rio/Igarapé/Riacho/Córrego|5957  |
|Poço artesiano                  |21506 |
|Cacimba/Cisterna/Poço           |10338 |
|Rede pública                    |139366|
|Sem informação                  |37395 |
+--------------------------------+------+

Distribuição original do Abastecimento de água:


+---------------+------+
|IN_AGUA_POTAVEL| count|
+---------------+------+
|              0|  7912|
|           null| 37395|
|              1|172318|
+---------------+------+



+--------------------+------+
|IN_AGUA_REDE_PUBLICA| count|
+--------------------+------+
|                   0| 40864|
|                null| 37395|
|                   1|139366|
+--------------------+------+



+----------------------+------+
|IN_AGUA_POCO_ARTESIANO| count|
+----------------------+------+
|                     0|151675|
|                  null| 37395|
|                     1| 28555|
+----------------------+------+



+---------------+------+
|IN_AGUA_CACIMBA| count|
+---------------+------+
|              0|164832|
|           null| 37395|
|              1| 15398|
+---------------+------+



+-----------------+------+
|IN_AGUA_FONTE_RIO| count|
+-----------------+------+
|                0|173303|
|             null| 37395|
|                1|  6927|
+-----------------+------+



+-------------------+------+
|IN_AGUA_INEXISTENTE| count|
+-------------------+------+
|                  0|177167|
|               null| 37395|
|                  1|  3063|
+-------------------+------+

É possível possuir mais de um local de abastecimento.


Quantidade de registros marcados com mais de um local de abastecimento:  11247


In [11]:
analise3 = analise3.withColumn('tipo_energia', F.when(F.col('IN_ENERGIA_REDE_PUBLICA') == 1, 'Rede pública').\
                         when(F.col('IN_ENERGIA_RENOVAVEL') == 1, 'Fontes de energia renováveis ou alternativas').\
                         when(F.col('IN_ENERGIA_INEXISTENTE') == 1, 'Não há energia elétrica').\
                         when(F.col('IN_ENERGIA_GERADOR_FOSSIL') == 1, 'Gerador movido a combustível fóssil').otherwise('Sem informação'))

In [12]:
analise3.groupBy('tipo_energia').count().show(truncate=False)

print('Distribuição original do Abastecimento de energia elétrica:')
for col in censo.columns:
    if col.startswith('IN_ENERGIA'):
        censo.groupBy(col).count().show()
        
# O count abaixo é apenas referente a um conjunto de combinações com o tipo de eneria. A quantidade pode ser maior ou menor a depender das combinações.
print('É possível possuir mais de um tipo de eneria.')
print('Quantidade de registros marcados com mais de um tipo de energia: ', censo.filter((F.col('IN_ENERGIA_RENOVAVEL') == 1) & ((F.col('IN_ENERGIA_GERADOR_FOSSIL') == 1) | (F.col('IN_ENERGIA_REDE_PUBLICA') == 1) | (F.col('IN_ENERGIA_INEXISTENTE') == 1))).count())

+--------------------------------------------+------+
|tipo_energia                                |count |
+--------------------------------------------+------+
|Não há energia elétrica                     |2585  |
|Rede pública                                |174594|
|Gerador movido a combustível fóssil         |1554  |
|Sem informação                              |37395 |
|Fontes de energia renováveis ou alternativas|1497  |
+--------------------------------------------+------+

Distribuição original do Abastecimento de energia elétrica:


+-----------------------+------+
|IN_ENERGIA_REDE_PUBLICA| count|
+-----------------------+------+
|                      0|  5636|
|                   null| 37395|
|                      1|174594|
+-----------------------+------+



+-------------------------+------+
|IN_ENERGIA_GERADOR_FOSSIL| count|
+-------------------------+------+
|                        0|178237|
|                     null| 37395|
|                        1|  1993|
+-------------------------+------+



+--------------------+------+
|IN_ENERGIA_RENOVAVEL| count|
+--------------------+------+
|                   0|176088|
|                null| 37395|
|                   1|  4142|
+--------------------+------+



+----------------------+------+
|IN_ENERGIA_INEXISTENTE| count|
+----------------------+------+
|                     0|177645|
|                  null| 37395|
|                     1|  2585|
+----------------------+------+

É possível possuir mais de um tipo de eneria.


Quantidade de registros marcados com mais de um tipo de energia:  2700


In [13]:
analise3 = analise3.withColumn('tipo_esgoto', F.when(F.col('IN_ESGOTO_REDE_PUBLICA') == 1, 'Rede pública').\
                         when(F.col('IN_ESGOTO_FOSSA_SEPTICA') == 1, 'Fossa septica').\
                         when(F.col('IN_ESGOTO_FOSSA_COMUM') == 1, 'Fossa rudimentar/comum').\
                         when(F.col('IN_ESGOTO_FOSSA') == 1, 'Fossa').\
                         when(F.col('IN_ESGOTO_INEXISTENTE') == 1, 'Não há esgotamento sanitário').otherwise('Sem informação'))

In [14]:
analise3.groupBy('tipo_esgoto').count().show(truncate=False)

print('Distribuição original do Esgoto sanitário:')
for col in censo.columns:
    if col.startswith('IN_ESGOTO'):
        censo.groupBy(col).count().show()
        
# O count abaixo é apenas referente a um conjunto de combinações com o tipo de esgoto. A quantidade pode ser maior ou menor a depender das combinações.
print('É possível possuir mais de um tipo de esgoto.')
print('Quantidade de registros marcados com mais de um tipo de esgoto: ', censo.filter((F.col('IN_ESGOTO_REDE_PUBLICA') == 1) & ((F.col('IN_ESGOTO_FOSSA_SEPTICA') == 1) | (F.col('IN_ESGOTO_FOSSA_COMUM') == 1) | (F.col('IN_ESGOTO_FOSSA') == 1))).count())

+----------------------------+------+
|tipo_esgoto                 |count |
+----------------------------+------+
|Não há esgotamento sanitário|6363  |
|Fossa septica               |38948 |
|Fossa rudimentar/comum      |31599 |
|Rede pública                |103320|
|Sem informação              |37395 |
+----------------------------+------+

Distribuição original do Esgoto sanitário:


+----------------------+------+
|IN_ESGOTO_REDE_PUBLICA| count|
+----------------------+------+
|                     0| 76910|
|                  null| 37395|
|                     1|103320|
+----------------------+------+



+-----------------------+------+
|IN_ESGOTO_FOSSA_SEPTICA| count|
+-----------------------+------+
|                      0|138704|
|                   null| 37395|
|                      1| 41526|
+-----------------------+------+



+---------------------+------+
|IN_ESGOTO_FOSSA_COMUM| count|
+---------------------+------+
|                    0|147634|
|                 null| 37395|
|                    1| 32596|
+---------------------+------+



+---------------+------+
|IN_ESGOTO_FOSSA| count|
+---------------+------+
|              0|106108|
|           null| 37395|
|              1| 74122|
+---------------+------+



+---------------------+------+
|IN_ESGOTO_INEXISTENTE| count|
+---------------------+------+
|                    0|173867|
|                 null| 37395|
|                    1|  6363|
+---------------------+------+

É possível possuir mais de um tipo de esgoto.


Quantidade de registros marcados com mais de um tipo de esgoto:  3575


In [15]:
analise3 = analise3.withColumn('descarte_lixo', F.when(F.col('IN_LIXO_QUEIMA') == 1, 'Queima').\
                         when(F.col('IN_LIXO_ENTERRA') == 1, 'Enterra').\
                         when(F.col('IN_LIXO_DESTINO_FINAL_PUBLICO') == 1, 'Leva a uma destinação final financiada pelo poder público').\
                         when(F.col('IN_LIXO_DESCARTA_OUTRA_AREA') == 1, 'Descarte em outra área').otherwise('Sem informação'))

In [16]:
analise3.groupBy('descarte_lixo').count().show(truncate=False)

print('Distribuição original de Destinação do lixo:')
for col in censo.columns:
    if col.startswith('IN_ESGOTO'):
        censo.groupBy(col).count().show()
        
# O count abaixo é apenas referente a um conjunto de combinações com o tipo de descarte do lixo. A quantidade pode ser maior ou menor a depender das combinações.
print('É possível possuir mais de um tipo de descarte do lixo.')
print('Quantidade de registros marcados com mais de um tipo de descarte do lixo: ', censo.filter((F.col('IN_LIXO_QUEIMA') == 1) & ((F.col('IN_LIXO_ENTERRA') == 1) | (F.col('IN_LIXO_DESTINO_FINAL_PUBLICO') == 1) | (F.col('IN_LIXO_DESCARTA_OUTRA_AREA') == 1))).count())

+---------------------------------------------------------+------+
|descarte_lixo                                            |count |
+---------------------------------------------------------+------+
|Leva a uma destinação final financiada pelo poder público|3913  |
|Descarte em outra área                                   |2999  |
|Queima                                                   |29769 |
|Enterra                                                  |1246  |
|Sem informação                                           |179698|
+---------------------------------------------------------+------+

Distribuição original de Destinação do lixo:


+----------------------+------+
|IN_ESGOTO_REDE_PUBLICA| count|
+----------------------+------+
|                     0| 76910|
|                  null| 37395|
|                     1|103320|
+----------------------+------+



+-----------------------+------+
|IN_ESGOTO_FOSSA_SEPTICA| count|
+-----------------------+------+
|                      0|138704|
|                   null| 37395|
|                      1| 41526|
+-----------------------+------+



+---------------------+------+
|IN_ESGOTO_FOSSA_COMUM| count|
+---------------------+------+
|                    0|147634|
|                 null| 37395|
|                    1| 32596|
+---------------------+------+



+---------------+------+
|IN_ESGOTO_FOSSA| count|
+---------------+------+
|              0|106108|
|           null| 37395|
|              1| 74122|
+---------------+------+



+---------------------+------+
|IN_ESGOTO_INEXISTENTE| count|
+---------------------+------+
|                    0|173867|
|                 null| 37395|
|                    1|  6363|
+---------------------+------+

É possível possuir mais de um tipo de descarte do lixo.


Quantidade de registros marcados com mais de um tipo de descarte do lixo:  6170


In [17]:
analise3 = analise3.select('NO_REGIAO', 'NO_UF', 'SG_UF', 'NO_MUNICIPIO', 'NO_MICRORREGIAO', 'NO_ENTIDADE', 'tipo_vinculo', 'local_func_escola',  'tipo_agua', 'tipo_energia', 'tipo_esgoto', 'descarte_lixo')
analise3 = analise3.toDF(*[c.upper() for c in analise3.columns])
completudeSchema(analise3)

Qtd. registros: 217625 | Quantidade de colunas:  12
root
 |-- NO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- NO_MICRORREGIAO: string (nullable = true)
 |-- NO_ENTIDADE: string (nullable = true)
 |-- TIPO_VINCULO: string (nullable = false)
 |-- LOCAL_FUNC_ESCOLA: string (nullable = false)
 |-- TIPO_AGUA: string (nullable = false)
 |-- TIPO_ENERGIA: string (nullable = false)
 |-- TIPO_ESGOTO: string (nullable = false)
 |-- DESCARTE_LIXO: string (nullable = false)



#### Tratamento das variáveis

- Remoção de numerações no nome da escola

In [18]:
print('Quantidade de registros que iniciam com numerações que não fazem parte do nome: ', analise3.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).count())

analise3.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).select("NO_ENTIDADE").show(truncate=False)

Quantidade de registros que iniciam com numerações que não fazem parte do nome:  1771


+--------------------------------------------------------+
|NO_ENTIDADE                                             |
+--------------------------------------------------------+
|29092191 - COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL |
|0101501 CIEP HENFIL                                     |
|0101803 EDI ANTONIO RAPOSO TAVARES                      |
|0101004 EM BENJAMIM CONSTANT                            |
|0101003 EM DARCY VARGAS                                 |
|0101006 EM FRANCISCO BENJAMIN GALLOTTI                  |
|0101005 EM GENERAL MITRE                                |
|0101008 EM MARECHAL ESPERIDIAO ROSAS                    |
|0101007 EM MARECHAL MASCARENHAS DE MORAES               |
|0101012 EM PROFESSORA LAURA SYLVIA MENDES PEREIRA       |
|0101009 EM PROFESSOR WALTER CARLOS DE MAGALHAES FRAENKEL|
|0101001 EM VICENTE LICINIO CARDOSO                      |
|0102503 CIEP AVENIDA DOS DESFILES II                    |
|0102502 CIEP AVENIDA DOS DESFILES I                    

In [19]:
# substituição das numerações por vázio
analise3 = analise3.withColumn("NOME_ENTIDADE", F.regexp_replace(F.col("NO_ENTIDADE"), "^(\d+\.?){3,} (- )?", ''))

print('Validando as modificações: ')
analise3.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).select("NO_ENTIDADE", "NOME_ENTIDADE").show(truncate=False)

Validando as modificações: 


+--------------------------------------------------------+------------------------------------------------+
|NO_ENTIDADE                                             |NOME_ENTIDADE                                   |
+--------------------------------------------------------+------------------------------------------------+
|29092191 - COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL |COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL    |
|0101501 CIEP HENFIL                                     |CIEP HENFIL                                     |
|0101803 EDI ANTONIO RAPOSO TAVARES                      |EDI ANTONIO RAPOSO TAVARES                      |
|0101004 EM BENJAMIM CONSTANT                            |EM BENJAMIM CONSTANT                            |
|0101003 EM DARCY VARGAS                                 |EM DARCY VARGAS                                 |
|0101006 EM FRANCISCO BENJAMIN GALLOTTI                  |EM FRANCISCO BENJAMIN GALLOTTI                  |
|0101005 EM GENERAL MITRE   

In [20]:
analise3.drop('NO_ENTIDADE')
pessoal.completudeSchema(analise3)

Qtd. registros: 217625 | Quantidade de colunas:  13
root
 |-- NO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- NO_MICRORREGIAO: string (nullable = true)
 |-- NO_ENTIDADE: string (nullable = true)
 |-- TIPO_VINCULO: string (nullable = false)
 |-- LOCAL_FUNC_ESCOLA: string (nullable = false)
 |-- TIPO_AGUA: string (nullable = false)
 |-- TIPO_ENERGIA: string (nullable = false)
 |-- TIPO_ESGOTO: string (nullable = false)
 |-- DESCARTE_LIXO: string (nullable = false)
 |-- NOME_ENTIDADE: string (nullable = true)



#### Escrita

In [21]:
writeSingleCsv(analise3, write_path)

('/home/tatiane/Documentos/censo_escolar2/projeto_2/analise3/analise3.csv',
 4.4367333549998875,
 5.267749419000211)

### Finalização do notebook

In [22]:
executionTime()

Tempo de execucao ate este ponto: 0:02:17.981846


In [23]:
sc.stop()